# 4. Collaborative Filtering - Item Based

In [2]:
# imports

import pandas as pd
import numpy as np
import json
import gzip

In [3]:
# define file paths

path_meta = 'original_data/meta-Utah.json.gz'
path_pivot_subset = 'data/subset.parquet'
path_reviews_subset = 'data/subset_reviews.parquet'
path_user_sim = 'data/subset_user_sim.parquet'
path_item_sim = 'data/subset_item_sim.parquet'

In [4]:
# import the item similarities

item_sim = pd.read_parquet(path_item_sim)

In [33]:
# import the reviews for the 10,000 subset of users

reviews = pd.read_parquet(path_reviews_subset)

### Helper functions

In [6]:
# define a function for reading the data using a generator

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

# define a function to get all the reviews for a user, sorted by rating in descending order

def get_user_rated_sorted(user_id, df):
    user_reviews = df[df['user_id'] == user_id]
    user_rated = dict(zip(user_reviews['gmap_id'], user_reviews['rating']))
    user_rated = pd.DataFrame(list(user_rated.items()), columns=['gmap_id', 'rating'])
    user_rated.sort_values(by='rating', ascending=False, inplace=True)
    return user_rated

# define a function to get a user's favorite places, favorites defined as being rated 4 or higher

def get_favorites(user_id, df):
  user_rated_sorted = get_user_rated_sorted(user_id, df)
  favorites = user_rated_sorted[user_rated_sorted['rating'] >= 4]
  return favorites

# define a function to get n most popular businesses, popular determined as 1) the most # of reviews and 2) highest average review

def get_popular(n, df):
  popular = df.groupby('gmap_id')['rating'].agg(['count','mean']).sort_values(by=['count','mean'], ascending=False)
  return popular.head(n)

# create a function to return the business name using the gmap_id

def get_business_name(gmap_id):
  meta_generator = parse(path_meta)
  for place in meta_generator:
    if place.get('gmap_id') == gmap_id:
      name = place['name']
      break
  return name

# create a function to return the business rating using the gmap_id

def get_business_rating(gmap_id):
  meta_generator = parse(path_meta)
  for place in meta_generator:
    if place.get('gmap_id') == gmap_id:
      avg_rating = place['avg_rating']
      break
  return avg_rating

In [29]:
get_user_rated_sorted('100518858506638839555', reviews)

,gmap_id,rating
46,0x8752880044f13fdd:0x908842aecab80a3f,5
14,0x87528bcc9cb4e93d:0xc3339f491d50180,5
25,0x8752890421f75c1b:0x395cf8820d09cf84,5
1,0x87528bcd22dedaf1:0x46fabacc23efe1cb,5
22,0x87528bedbcd1febf:0x258ad68cc64a1d81,5
21,0x8752f4e9dd9d9d5f:0x5d6f6cc2c8bebb8d,5
20,0x87528df1707f2019:0x7d5773c9b9fbbd0d,5
19,0x8752f479e557b67d:0x9e117cd5f64f9c6,5
30,0x875261a7924c019b:0x4a64591038379f9f,5
34,0x874d9a896798ded9:0xdc9ca4adc01529e3,5


### Recommender function

In [31]:
def item_based_recommendations(user_id, df, item_sim, n_recs):
  """
  Accepts the following parameters and returns a DataFrame containing n_recs recommendations.

  user_id = user_id
  df = the pivoted table
  item_sim = the item similarities matrix
  n_recs = the number of recommendations desired
  """

  # get the list of what the user already rated
  user_rated_sorted = get_user_rated_sorted(user_id, df)

  # get favorites for user
  user_favs = get_favorites(user_id, df)

  # create an empty dictionary to add recommendations to
  recs_dict = {}

  # for each favorite place in the user's list of favorite place:
  for fav in user_favs.values:
    fav = fav[0]

    # get the similar places with cosine similarity > 0
    user_item_sims = item_sim.loc[fav].sort_values(ascending=False)[1:]
    similar_places = user_item_sims[user_item_sims.values > 0]

    # for each of the similar places:
    for index, value in similar_places.items():
      place = index

      # if they are not in the list of places the user has already rated
      if place not in user_rated_sorted and place not in recs_dict:

      # then add the place's gmap_id and the similarity, to the dictionary.
        recs_dict[place] = similar_places[place]

      # if the place is already in the dictionary, then add the cosine similarity to the existing similarity
      if place in recs_dict:
        recs_dict[place] += similar_places[place]

  # turn the dictionary into a dataframe
  recs_df = pd.DataFrame(list(recs_dict.items()), columns=['gmap_id','similarity'])

  # sort the dataframe by the total cosine similarity, from highest to lowest
  recs_df.sort_values(by='similarity', ascending=False, inplace=True)

  # narrow down the recommendation to n * 2 by using .head
  recs_df = recs_df.head(n_recs*2)

  # for each place, add the business name
  recs_df['name'] = recs_df['gmap_id'].apply(get_business_name)

  # for each place, add the average rating
  recs_df['avg_rating'] = recs_df['gmap_id'].apply(get_business_rating)

  # sort by 1) similarity, 2) avg_rating
  recs_df.sort_values(by=['similarity','avg_rating'], ascending=False, inplace=True)

  # remove businesses that have an average rating under 3.5 stars
  recs_df = recs_df[recs_df['avg_rating'] > 3.5]

  # limit the length using head of n_recs
  recs = recs_df.head(n_recs)

  # calculate the difference between the length of list of recommendation and n_recs
  diff = n_recs - recs['gmap_id'].count()
  print(diff)

  # if the difference is 0, return the list of recommendations
  if diff == 0:
    return recs

  # else create an additional list of recommendations using get_popular, with n = the difference
  else:
    popular = get_popular(diff, df)

    # name the columns and then resent index
    popular.columns = ['similarity','avg_rating']
    popular.reset_index(inplace=True)

    # add the business name to the df
    popular['name'] = popular['gmap_id'].apply(get_business_name)

    # reorder the columns to match the recs df
    popular = popular[['gmap_id','similarity','name','avg_rating']]


  # add the additional list to the bottom of the original list of recommendations
    recs = pd.concat([recs, popular], ignore_index=True)

  return recs


In [32]:
# test the function 1

item_based_recommendations('100518858506638839555', reviews , item_sim, 10)

0


,gmap_id,similarity,name,avg_rating
159,0x87528a41510c8e8f:0x18d1e1bbde2dfd1f,8.033194,Roses Auto Body,4.5
106,0x8752f514bcf6c361:0x4acf33126073d429,8.033194,Healing Mountain Massage School (Salt Lake Cit...,4.3
88,0x87528a035ca1ca6f:0xf5828d533f0cb8b1,8.033194,"Sundance Behavioral Resources, LLC",3.6
99,0x8752f569a04d64bf:0x2b7e577cda98e81a,7.033194,Ensign Wholesale Floral,4.1
340,0x8752f479e557b67d:0x9e117cd5f64f9c6,5.690040,90.9fm KRCL,4.8
661,0x87528bcc9cb4e93d:0xc3339f491d50180,4.651843,Maverik,4.4
883,0x87528bcc833a38dd:0x5c0b0e80fafadd57,4.271734,Maverik,3.7
57,0x87528994f27c0929:0x5e328e3daa93f122,2.764877,FirstMed Urgent Care - Murray,3.8
957,0x87528bedbcd1febf:0x258ad68cc64a1d81,2.679980,Hidden Peaks | Coffee + Roasting Company,4.8
1986,0x87528bdc2128c417:0x64ea50e8f0bbbb33,2.413833,Millrace Park,4.3


In [9]:
# test the function 2

item_based_recommendations('12345', reviews , item_sim, 10)

/var/folders/bh/crnltg950_n8l07mfm97qdtc0000gn/T/ipykernel_52119/1187715242.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  recs = pd.concat([recs, popular], ignore_index=True)


,gmap_id,similarity,name,avg_rating
0,0x8752879236e85383:0xbaf443f16b40940c,467,Loveland Living Planet Aquarium,4.616702
1,0x87528e94d44e7fc5:0x42b1cfae697ba4a6,463,Jordan Landing,4.278618
2,0x8752f508b7dc56a9:0x13d77d6e854d7e79,461,City Creek Center,4.501085
3,0x8752f51b7961dc89:0x73f1a804eb909466,456,Hogle Zoo,4.552632
4,0x8752872f3f4b153b:0xd8bb5e82808eedbd,453,IKEA,4.278146
5,0x875289bef87fcb6b:0x9e865a4dadee3648,441,Fashion Place,4.328798
6,0x87528c9d1d641321:0x90cf6256d98a62eb,436,Valley Fair Mall,3.830275
7,0x8752f50023b5e105:0xbf4b78f58640694f,429,The Gateway,4.065268
8,0x8752ff18c387e561:0xf3256e9df9ab3603,417,Lagoon Amusement Park,4.342926
9,0x8752f54425332f67:0x354cd3a2499141d0,365,Liberty Park,4.641096


In [34]:
# test the function 3

item_based_recommendations('111717473911684632928', reviews , item_sim, 50)

0


,gmap_id,similarity,name,avg_rating
0,0x8752f796b56c557d:0x54742b5acc6749f6,2.000000,"Christine Packard, PAC | Utah Family Practice",4.9
4,0x875303405f87384b:0x1a63d54bda594a90,2.000000,"Sheryl Bingham, FNP | Medical Weight Loss Expert",4.9
1,0x87530f257acbaf01:0xa9cd9057d50e7927,2.000000,Sawyer Lock and Key,4.9
3,0x87530ec0a9b4b56f:0x9827798770b3895d,2.000000,Shaidz Hair Salon,4.6
2,0x87530f23e3ee22f9:0xf7cc14b639406420,2.000000,Mark-A-Newt Appliance Specialists,4.5
6,0x8753bad2cb88cbad:0x58c01b9cbb9ffd25,2.000000,Magpie Campground,4.1
5,0x8753bad6f0c99bd3:0x8e5cc8a326c852ba,2.000000,Botts Campground,4.0
1885,0x8752890fa6ea0757:0x2cc1b06be7651e2e,1.861010,Momentum Electric,5.0
1883,0x874d9715e96fdf95:0xca64169390b79798,1.861010,Treeline,4.8
1875,0x874d9b3219acad3d:0x34b4fc26e6a5b6db,1.861010,Simple Elegance Rock Shop,4.8
